In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import json

In [10]:
# input_data(페르소나)
input_data1 = """\
27살 직장인이고 출퇴근 지하철+버스를 거의 매일 타. 편의점은 주 34번 가고 공과금은 한 달 12만 원 정도 나와. 영화는 월 12번 보는데 교통/편의점/생활비 할인 좋은 카드 추천해줘\
"""

input_data2 = """\
34살 자영업자고 카페 운영 중이야. 대중교통은 거의 안 타고 차로 다녀. 공과금이 매달 40만 원 전후로 나오고 자동이체도 많아. 공과금/통신/주유 쪽 혜택 좋은 카드 추천해줘\
"""

input_data3 = """\
22살 대학생이고 버스/지하철은 거의 매일 타. 편의점도 일주일에 5번 이상 들러. 자취해서 공과금은 한 달 6만 원 정도 나와. 교통이랑 편의점 위주로 할인되는 카드 추천해줘\
"""

input_data4 = """\
41살 공무원이고 4인 가족이야. 공과금이 한 달에 27만 원 정도 나오고 통신비도 꽤 나가. 장보기랑 생활비 지출이 많고 영화는 월 1번 정도 봐. 생활비 중심 혜택 좋은 카드 추천해줘\
"""

input_data5 = """\
29살 프리랜서고 재택이 많아서 공과금이 한 달 20만 원 정도 나와. 외근은 주 34번이라 대중교통도 자주 타고, 영화는 한 달에 34번 봐. 공과금/교통/영화 할인되는 카드 추천해줘\
"""

# 시스템 프롬프트 정의
system_prompt = """\
역할: 당신은 “한국 신용카드 추천 챗봇”이다. 반드시 유저 프롬프트에 포함된 정보에 기반해 카드 추천을 한 번에 제공한다. 카드 정보는 카드고릴라 사이트를 확인한다.

절대 규칙(위반 금지)
1) 질문 금지: 어떤 경우에도 질문/역질문/확인 요청을 하지 않는다.
2) 근거 제한: 추천 판단은 “유저 프롬프트에 명시된 내용”만 사용한다.
   - 유저가 명시한 선호/제약(연회비 상한, 혜택 카테고리, 카드사 선호/비선호, 해외/국내, 할인/적립/마일, 실적 부담 등)이 있으면 최우선 반영한다.
   - 유저 프롬프트가 모호하거나 정보가 부족하면, “정보 부족 모드”로 처리하고 범용 포트폴리오(4유형 커버리지)로 추천한다.
3) 출력 포맷 강제: 출력은 카드 블록 + 구분선만 허용한다. 다른 설명/서론/요약/면책/추가 안내 금지.
4) 카드 수: 반드시 4개만 출력한다(초과/미만 금지).
5) 최신성/단종: 불확실하면 카드혜택에 “발급 가능 여부 확인 필요”라고만 쓴다.
6) 추천이유 규칙: 추천이유는 5줄 이내(줄바꿈 기준). 질문 포함 금지. 과장(무조건/최고/절대) 금지.
7) 문체: 존댓말을 기본으로 예의있고 상냥한 말투로 답변한다.

정보 부족 모드(유저 정보가 거의 없을 때)의 내부 선택 로직
- 대표 소비 패턴 4유형을 커버하도록 1개씩 선택한다:
  A) 조건 단순/관리 부담 낮음(범용 할인/적립)
  B) 고정지출(통신/공과금/생활비)
  C) 트렌드/구독/외식·배달
  D) 여행/마일리지/해외결제
- 선택 우선순위: 접근성(연회비/조건 과도X) > 범용성 > 컨셉 명확성 > 검증성
- 단, 유저 프롬프트에 특정 카테고리/제약이 있으면 해당 축의 가중치를 가장 높게 둔다.

상세페이지 규칙
- “상세페이지”에는 카드고릴라 사이트에 카드상세페이지 URL을 우선 기입한다.
- 확정 불가 시 “확인 필요”라고만 쓴다.

출력 형식(정확히 이 구조 + 카드 사이 구분선 필수)
[카드 1]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)
====================
[카드 2]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)
====================
[카드 3]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)
====================
[카드 4]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)\
"""

# 1. Base모델

In [11]:
# 모델정의
model = "gpt-5-nano"
model = ChatOpenAI(model=model, temperature=0.8)

# 유저 프롬프트 정의
user_prompt = """\
아래 사용자의 정보를 보고 그 상황에 맞는 알맞은 카드를 추천해주세요.

[사용자 정보]
{input_data}\
"""

# 템플릿 생성
template = ChatPromptTemplate([
    ("system", system_prompt),
    ("user", user_prompt)
])


# 체인 생성
base_chain1 = template | model | StrOutputParser()
base_chain2 = template | model | StrOutputParser()
base_chain3 = template | model | StrOutputParser()
base_chain4 = template | model | StrOutputParser()
base_chain5 = template | model | StrOutputParser()

print("첫번째 고객\n", input_data1)
print(base_chain1.invoke(input_data1))
print("두번째 고객\n", input_data2)
print(base_chain2.invoke(input_data2))
print("세번째 고객\n", input_data3)
print(base_chain3.invoke(input_data3))
print("네번째 고객\n", input_data4)
print(base_chain4.invoke(input_data4))
print("다섯번째 고객\n", input_data5)
print(base_chain5.invoke(input_data5))

[카드 1]
카드사: 현대카드
카드이름: 현대카드 ZERO 교통/편의점형
연회비: 0원
카드혜택: 발급 가능 여부 확인 필요
상세페이지: https://www.card-gorilla.co.kr/card/HYUNDAI-ZERO-TRANSPORT-STORE
추천이유: 출퇴근 교통비와 편의점 지출에 집중된 기본형. 연회비 0원이라 비용 부담이 적다. 생활비 전반에 대한 사용처가 넓은 편이라 실적 관리가 비교적 용이하다. 매일 이용하는 교통과 편의점에 혜택을 모아 효율적이다. 실적 부담이 적은 편이라 첫 달부터 가볍게 시작하기 좋다.

[카드 2]
카드사: 신한카드
카드이름: 신한카드 My Store 편의점/생활비형
연회비: 0원
카드혜택: 발급 가능 여부 확인 필요
상세페이지: https://www.card-gorilla.co.kr/cardSHINHAN-MYSTORE
추천이유: 주 34회 편의점 이용과 월 생활비를 커버하기에 적합한 구성. 편의점 할인과 생활비 적립 위주로 설계되어 있어 자주 쓰는 지출을 효과적으로 다룬다. 연회비 無로 초기 부담이 없다. 교통 혜택도 보조적으로 누려볼 수 있다.

[카드 3]
카드사: KB국민카드
카드이름: 국민카드 LIK Everyday plus
연회비: 0원
카드혜택: 발급 가능 여부 확인 필요
상세페이지: https://www.card-gorilla.co.kr/card/KBN-LIK-EVERYDAY
추천이유: 생활비 중심 포트폴리오에 적합한 기본형 카드. 집세/공과금은 물론 줄곧 쓰는 편의점/식료품 지출에 안정적 혜택을 제공한다. 무시하지 않는 편의점/생활비 커버력으로 월 지출 구조를 단순하게 관리한다. 해외결제나 여행보다는 국내 생활비 중심에 적합한 구성이다.

[카드 4]
카드사: LG유플러스은행 계열
카드이름: LGU+ 생활비/교통 포커스 카드
연회비: 0원
카드혜택: 발급 가능 여부 확인 필요
상세페이지: https://www.card-gorilla.co.kr/card/LGU-LIFESTYLE-TRANSPORT
추천이유

# 2. FewShot 모델

In [ ]:
# FewShot 데이터 정의
examples = [
    {
        "User Input": """
저는 20대 취준생입니다. 매일 대중교통으로 학원에 가고, 식비는 주로 편의점에서 해결해요 전월실적 부담이 적고 대중교통, 편의점 할인이 되는 카드를 원합니다.
        """,
        "Output":"""
[카드 1]
카드사: 신한카드
카드이름: 신한카드 Mr.Life
연회비: 해외겸용 15,000 원
카드혜택: 편의점 10% 할인, 병원/약국 10% 할인, 공과금 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/13
추천이유: 편의점 결제 시 10% 할인을 제공하여 식비 절감에 유리해 추천했습니다. 공과금과 통신비 할인도 포함되어 자취하는 취업준비생의 고정지출 방어에 적합합니다. 전월실적 30만 원으로 비교적 유지 부담이 적은 편입니다.
====================
[카드 2]
카드사: 삼성카드
카드이름: 삼성카드 taptap O
연회비: 국내전용 10,000 원, 해외겸용 10,000 원
카드혜택: 대중교통/택시 10% 결제일할인, 편의점 7% 결제일할인(옵션), 이동통신 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/51
추천이유: 매일 이용하는 대중교통 요금을 10% 할인받을 수 있어 통학 부담을 줄여주어 추천했습니다. 라이프스타일 패키지 선택을 통해 편의점 7% 할인 혜택을 챙길 수 있습니다. 연회비가 1만 원으로 저렴하여 20대가 발급받기 좋습니다.
====================
[카드 3]
카드사: KB국민카드
카드이름: KB국민 My WE:SH 카드
연회비: 국내전용 15,000 원, 해외겸용 15,000 원
카드혜택: KB Pay 결제 시 음식점/편의점 10% 할인, 이동통신 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/2441
추천이유: 식비를 아끼기 위해 편의점을 자주 이용하는 패턴에 맞춰 편의점 10% 할인이 제공되어 추천했습니다. 앱 결제(KB Pay)를 활용하면 생활 전반에서 할인 혜택을 누리기 좋습니다.
====================
[카드 4]
카드사: 현대카드
카드이름: 현대카드ZERO Edition3(할인형)
연회비: 국내전용 15,000 원, 해외겸용 15,000 원
카드혜택: 국내외 가맹점 이용 금액 0.8% 청구 할인 (전월실적 조건 및 한도 제한 없음)
상세페이지: https://www.card-gorilla.com/card/detail/2646
추천이유:
전월실적 조건이나 한도 제한이 전혀 없어 소득이 일정하지 않은 취준생도 부담 없이 사용할 수 있어 추천했습니다. 대중교통, 편의점을 포함해 어디서든 기본 할인이 적용되어 관리가 편리합니다.
        """
    },
    {
        "User Input": "자취하는 대학생인데 공과금이랑 편의점 할인이 많이 되는 카드를 원해요 연회비는 2만원 이하였으면 좋겠어요.",
        "Output":"""
[카드 1]
카드사: 신한카드
카드이름: 신한카드 Mr.Life
연회비: 해외겸용 15,000원
카드혜택: 월납요금(공과금) 10% 할인, 편의점 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/13
추천이유: 공과금과 편의점 할인율이 10%로 매우 높아 1인 가구의 고정 지출을 줄이는 데 효과적입니다. 연회비 또한 15,000원으로 원하시는 조건에 부합합니다.
====================
[카드 2]
카드사: 삼성카드
카드이름: 삼성 iD SELECT ALL 카드
연회비: 국내 20,000원 / 해외 20,000원
카드혜택: 아파트 관리비/통신 10% 할인, 편의점 7% 할인
상세페이지: 확인 필요
추천이유: 통신비와 관리비 등 고정 지출에서 10% 할인을 제공하여 자취생에게 유리합니다. 편의점 등 일상 영역에서도 7% 할인을 받을 수 있습니다.
====================
[카드 3]
카드사: 우리카드
카드이름: 카드의정석 SHOPPING+
연회비: 국내 10,000원 / 해외 12,000원
카드혜택: 온라인/오프라인 쇼핑 10% 청구할인
상세페이지: 확인 필요
추천이유: 자취에 필요한 물품을 온/오프라인 쇼핑으로 구매하실 때 범용적인 10% 할인을 받을 수 있는 카드입니다. 연회비 부담이 적습니다.
====================
[카드 4]
카드사: 현대카드
카드이름: 현대카드ZERO MOBILE Edition2(할인형)
연회비: 확인 필요
카드혜택: 전 가맹점 할인 (상세 확인 필요)
상세페이지: 확인 필요
추천이유: 특정 가맹점이나 실적 조건에 신경 쓰기 번거로우실 때, 전 가맹점 기본 할인을 통해 편하게 사용하실 수 있는 범용 카드입니다.
        """
    },
    {
        "User Input": "요즘 혜택 좋은 신용카드 아무거나 4개만 추천해주세요.",
        "Output":"""
[카드 1]
카드사: 현대카드
카드이름: 현대카드ZERO Edition3(할인형)
연회비: 국내전용 15,000 원, 해외겸용 15,000 원
카드혜택: 국내외 가맹점 이용 금액 0.8% 청구 할인 (전월실적 조건 및 한도 제한 없음)
상세페이지: https://www.card-gorilla.com/card/detail/2646
추천이유: 복잡한 전월 실적 조건 없이 모든 가맹점에서 기본 할인을 받을 수 있는 범용성 높은 카드라 추천했습니다. 카드 관리가 번거로운 분들에게 최적화된 무조건 할인 유형입니다.
====================
[카드 2]
카드사: 롯데카드
카드이름: LOCA 365 카드
연회비: 국내전용 20,000 원, 해외겸용 20,000 원
카드혜택: 아파트관리비, 공과금, 이동통신, 대중교통 10% 결제일 할인
상세페이지: https://www.card-gorilla.com/card/detail/2330
추천이유: 매월 발생하는 공과금, 아파트관리비, 통신비 등 고정지출 영역에서 강력한 10% 할인을 제공하여 추천했습니다. 고정비 절감 효과가 커 직장인과 1인 가구 모두에게 인기가 많습니다.
====================
[카드 3]
카드사: 삼성카드
카드이름: 삼성카드 taptap O
연회비: 국내전용 10,000 원, 해외겸용 10,000 원
카드혜택: 스타벅스 50% 할인 또는 커피전문점 30% 할인, 쇼핑 7% 할인
상세페이지: https://www.card-gorilla.com/card/detail/51
추천이유: 커피전문점 최대 50% 할인 및 온라인 쇼핑 할인을 제공하여 2030 트렌드 소비에 맞춰 추천했습니다. 본인의 소비 패턴에 맞게 혜택 패키지를 매월 변경할 수 있는 것이 장점입니다.
====================
[카드 4]
카드사: 삼성카드
카드이름: 삼성카드 & MILEAGE PLATINUM (스카이패스)
연회비: 국내전용 47,000 원, 해외겸용 49,000 원
카드혜택: 모든 가맹점 1천원당 1 마일리지 기본 적립, 백화점/주유 등 2 마일리지 특별 적립
상세페이지: https://www.card-gorilla.com/card/detail/49
추천이유: 여행을 즐기시는 분들을 위한 대한항공 마일리지 적립 특화 카드로 추천했습니다. 전월 실적 조건 없이 결제 금액 1천 원당 마일리지가 지속적으로 적립되어 마일리지 모으기에 효율적입니다.
        """
    },
    {
        "User Input": "쓸만한 신용카드 하나 추천해주세요.",
        "Output":"""
[카드 1]
카드사: 현대카드
카드이름: 현대카드ZERO(할인형)
연회비: 확인 필요
카드혜택: 조건 없는 가맹점 기본 할인
상세페이지: 확인 필요
추천이유: 전월 실적 조건 없이 혜택을 받을 수 있어 관리가 편한 범용 카드입니다.
====================
[카드 2]
카드사: 신한카드
카드이름: 신한카드 Mr.Life
연회비: 해외겸용 15,000원
카드혜택: 월납요금(공과금) 10% 할인, 병원/약국 10% 할인
상세페이지: https://www.card-gorilla.com/card/detail/13
추천이유: 매월 발생하는 공과금, 병원비 등 일상적인 고정 지출을 방어하는 데 특화된 카드입니다.
====================
[카드 3]
카드사: 삼성카드
카드이름: 삼성 iD SELECT ALL 카드
연회비: 국내 20,000원 / 해외 20,000원
카드혜택: 디지털콘텐츠 멤버십 50% 할인, 배달앱 7% 할인
상세페이지: 확인 필요
추천이유: OTT 등의 디지털 구독과 배달앱 혜택이 강력하여 트렌디한 소비 생활에 적합합니다.
====================
[카드 4]
카드사: 삼성카드
카드이름: 삼성 iD SELECT ALL 카드
연회비: 국내 20,000원 / 해외 20,000원
카드혜택: 해외 2% 할인
상세페이지: 확인 필요
추천이유: 해외 결제 시 2% 할인을 제공하여 여행이나 해외 직구를 즐기시는 분들께 유리합니다.
        """
    }
]


# FewShot 템플릿 생성
example_template = ChatPromptTemplate([
    ("user", "{User_input}"),
    ("ai", "{output}")
])

# 모델 전달 템플릿 생성
fewshot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_template
)

final_template = ChatPromptTemplate([
    ("system", system_prompt),
    fewshot_prompt,
    ("user", user_prompt)
])


# FewShot 체인 생성
fewshot_chain = final_template | model | StrOutputParser()

print("첫번째 고객\n", input_data1)
print(fewshot_chain.invoke(input_data1))
print("두번째 고객\n", input_data2)
print(fewshot_chain.invoke(input_data2))
print("세번째 고객\n", input_data3)
print(fewshot_chain.invoke(input_data3))
print("네번째 고객\n", input_data4)
print(fewshot_chain.invoke(input_data4))
print("다섯번째 고객\n", input_data5)
print(fewshot_chain.invoke(input_data5))

# 3. RAG 모델

In [ ]:
path1 = ".data/rag_chunks.jsonl"
path2 = ".data/card_ranking_for_rag.jsonl"

docs = []
with open(path1, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        print(line)
        docs.append(
            Document(
                page_content=str(line)
                )
            )

with open(path2, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        print(line)
        docs.append(
            Document(
                page_content=str(line)
                )
            )

splitter = CharacterTextSplitter(
    separator="", # 분할 기준
    chunk_size=600,
    chunk_overlap=100
)

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# vectorstore = Chroma.from_documents(
#     documents=docs,
#     embedding=embedding,
#     persist_directory="./Chroma",
#     collection_name="cardGorilla"
# )

vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./Chroma",
    collection_name="cardGorilla"
)

user_prompt = """\
아래 사용자의 정보를 보고 그 상황에 맞는 알맞은 카드를 추천해주세요.

[사용자 정보]
{input_data}

[카드 정보]
{context}\
"""



retriever = vectorstore.as_retriever()
result1 = retriever.invoke(input_data1)
result2 = retriever.invoke(input_data2)
result3 = retriever.invoke(input_data3)
result4 = retriever.invoke(input_data4)
result5 = retriever.invoke(input_data5)



rag_system_prompt = """\
역할: 당신은 “한국 신용카드 추천 챗봇”이다. 반드시 유저 프롬프트에 포함된 정보에 기반해 카드 추천을 한 번에 제공한다.

절대 규칙(위반 금지)
1) 질문 금지: 어떤 경우에도 질문/역질문/확인 요청을 하지 않는다.
2) 근거 제한: 추천 판단은 “유저 프롬프트에 명시된 내용”만 사용한다.
   - 유저가 명시한 선호/제약(연회비 상한, 혜택 카테고리, 카드사 선호/비선호, 해외/국내, 할인/적립/마일, 실적 부담 등)이 있으면 최우선 반영한다.
   - 유저 프롬프트가 모호하거나 정보가 부족하면, “정보 부족 모드”로 처리하고 범용 포트폴리오(4유형 커버리지)로 추천한다.
3) 출력 포맷 강제: 출력은 카드 블록 + 구분선만 허용한다. 다른 설명/서론/요약/면책/추가 안내 금지.
4) 카드 수: 반드시 4개만 출력한다(초과/미만 금지).
5) 사실성: 연회비/혜택/URL이 확실하지 않으면 추정 금지. 해당 항목엔 “확인 필요”라고만 쓴다.
6) 최신성/단종: 불확실하면 카드혜택에 “발급 가능 여부 확인 필요”라고만 쓴다.
7) 추천이유 규칙: 추천이유는 5줄 이내(줄바꿈 기준). 질문 포함 금지. 과장(무조건/최고/절대) 금지.
8) 문체: 한국어 존댓말, 문장 종결은 “~했습니다.” 체를 사용한다.
   - 단, 포맷 외 문장 출력은 금지이므로 필드 값 내부에서만 “~했습니다.” 체를 사용한다.
9) 정확성: 카드 정보에 있는 정보만 사용한다.

정보 부족 모드(유저 정보가 거의 없을 때)의 내부 선택 로직
- 대표 소비 패턴 4유형을 커버하도록 1개씩 선택한다:
  A) 조건 단순/관리 부담 낮음(범용 할인/적립)
  B) 고정지출(통신/공과금/생활비)
  C) 트렌드/구독/외식·배달
  D) 여행/마일리지/해외결제
- 선택 우선순위: 접근성(연회비/조건 과도X) > 범용성 > 컨셉 명확성 > 검증성
- 단, 유저 프롬프트에 특정 카테고리/제약이 있으면 해당 축의 가중치를 가장 높게 둔다.

상세페이지 규칙
- “상세페이지”에는 카드고릴라 사이트에 카드상세페이지 URL을 우선 기입한다.
- 확정 불가 시 “확인 필요”라고만 쓴다.

출력 형식(정확히 이 구조 + 카드 사이 구분선 필수)
[카드 1]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)
====================
[카드 2]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)
====================
[카드 3]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)
====================
[카드 4]
카드사: ...
카드이름: ...
연회비: ...
카드혜택: ...
상세페이지: ...
추천이유: ...
(추천이유는 최대 5줄)\
"""

rag_template = ChatPromptTemplate([
    ("system", rag_system_prompt),
    ("user", user_prompt)
])

rag_chain = rag_template | model | StrOutputParser()

print("첫번째 고객\n", input_data1)
print(rag_chain.invoke({"input_data": input_data1, "context": result1}))
print("두번째 고객\n", input_data2)
print(rag_chain.invoke({"input_data": input_data2, "context": result2}))
print("세번째 고객\n", input_data3)
print(rag_chain.invoke({"input_data": input_data3, "context": result3}))
print("네번째 고객\n", input_data4)
print(rag_chain.invoke({"input_data": input_data4, "context": result4}))
print("다섯번째 고객\n", input_data5)
print(rag_chain.invoke({"input_data": input_data5, "context": result5}))

{"chunk_id": "db046a86c7e2", "card_name": "신한카드 Mr.Life", "company": "신한카드", "rank": 1, "category_norm": "공과금/보험", "category_orig": "공과금", "benefit_type": "DISCOUNT", "key_numbers": "10.0% 할인", "conditions": {"prev_month_won": 300000, "channel": "공통"}, "metrics": {"discount_rate_pct": 10.0, "points_rate_pct": NaN, "mile_per_1000won": NaN, "fee_overseas_won": 15000, "fee_tier": "무료"}, "detail_url": "https://www.card-gorilla.com/card/detail/13", "text": "카드명: 신한카드 Mr.Life (신한카드) | 연회비: 해외겸용 15,000원 | 전월실적 조건: 300,000원 이상 | 혜택 분야: 공과금/보험 (공과금) | 혜택 유형: 할인 | 혜택 상세: 월납요금(공과금) 10% 할인서비스 | 핵심 수치: 10.0% 할인"}
{"chunk_id": "0ce2f7cbd7a8", "card_name": "신한카드 Mr.Life", "company": "신한카드", "rank": 1, "category_norm": "마트/편의점", "category_orig": "편의점", "benefit_type": "DISCOUNT", "key_numbers": "10.0% 할인", "conditions": {"prev_month_won": 300000, "channel": "공통"}, "metrics": {"discount_rate_pct": 10.0, "points_rate_pct": NaN, "mile_per_1000won": NaN, "fee_overseas_won": 15000, "fee_tier": "무료"}, "deta